In [14]:
import cv2
import glob
from scipy.spatial import distance
import math
import pickle
import numpy as np
import pandas as pd
from utility import *
from direction_change_distances import direction_change_distances
from sklearn.model_selection import train_test_split
from sktime.classification.hybrid import HIVECOTEV2


In [15]:
from sklearn.metrics import accuracy_score
from matplotlib.font_manager import json_load
#import sys
#stdout = sys.stdout

In [135]:
# Read in csv file with video meta data
# each row needs to have following info: file path, video number, object, track
video_files = pd.read_csv('E:\\Sky360_videos\\video_files.csv')
print("finished")

finished


In [19]:
"""Run this cell for each object and attach numbers to dist_list,direction_change_list_cleaned and y_in accordingly. These 
lists are concatenated in the next cell."""

Object = "Plane" # Plane or Bird or Plant
sequence_length = 30 # Sequence length to train on

"""For a given object get a list of distances [px] and direction changes [°] calculated between consecutive images. This
will produce a warning for instances when the magnitude of a direction vector is 0. Will be dealt with in the following
lines."""
dist_list,direction_change_list = direction_change_distances(video_files,Object,sequence_length)

## Concatenation of input variables

"""direction_change_list contains NaNs here and there for the instances described above. Here it makes sense to just
replace nans with 0."""
direction_change_list_cleaned = clean_nans(direction_change_list)

# these are the object labels for model training
y_in = np.array([Object for _ in range(int(len(dist_list)/sequence_length))], dtype='<U9')

# little test if input training data and labels have equal lengths
len(dist_list)/sequence_length == len(y_in)
print("run complete")

run complete


In [20]:
"""concatenate model input data (dist_list, direction_change_list_cleaned) of different objects""" 

# input for model training
dist_list_all = list(dist_list) + list(dist_list2) + list(dist_list3)
direction_change_list_all = list(direction_change_list_cleaned) + list(direction_change_list_cleaned2) + list(direction_change_list_cleaned3)
# labels for model training
y = np.concatenate((y_in, y_in2,y_in3), axis=0)

In [21]:
print(len(y_in))
print(len(y_in2))
print(len(y_in3))

152
143
160


In [22]:
# Prepare input dataframe needed for the model
# First input variable: directional changes [px]
d = {}
n = 0
for i in range(0,len(dist_list_all)):
    if i % sequence_length == 0:
      d[n] = pd.Series(direction_change_list_all[i:i+sequence_length])
      n += 1
df = pd.Series(d).to_frame('Direction_changes')

# 2nd input variable: distances [°]
d = {}
n = 0
for i in range(0,len(dist_list_all)):
    if i % sequence_length == 0:
      d[n] = pd.Series(dist_list_all[i:i+sequence_length])
      n += 1
df2 = pd.Series(d).to_frame('Distances')

# Concatenate different input variables to one input dataframe
X = pd.concat([df, df2], axis=1)

In [23]:
# Little check: X.size divided by number of variables should equal y.size
print(X.size/2)
print(y.size)

455.0
455


In [24]:
# create datasets for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y)

Below the time series classification algo can be chosen, which will be used for training

In [25]:
# define the model
# check following link for the multitude of available algos: 
# https://www.sktime.org/en/stable/api_reference/classification.html

# here we just use the default values for now, has potential for improvements
classifier = HIVECOTEV2(
    stc_params={"n_shapelet_samples": 1000}, 
    drcif_params={"n_estimators": 25},
    arsenal_params={"n_estimators": 10},
    tde_params={"n_parameter_samples": 100},
    verbose=0,
)


In [32]:
#CNN classifier
from sktime.classification.deep_learning.cnn import CNNClassifier
classifier = CNNClassifier(n_epochs=20,batch_size=4)  

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:808: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


CNNClassifier(batch_size=4, n_epochs=20)

In [48]:
#FCN classifier
from sktime.classification.deep_learning.fcn import FCNClassifier
classifier = FCNClassifier(n_epochs=20,batch_size=4)  

In [60]:
#MLP classifier
from sktime.classification.deep_learning.mlp import MLPClassifier
classifier = MLPClassifier(n_epochs=20,batch_size=4)  

In [68]:
#MUSE classifier
from sktime.classification.dictionary_based import MUSE
classifier = MUSE(window_inc=4, use_first_order_differences=False)

In [72]:
#TemporalDictionaryEnsemble
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble
classifier = TemporalDictionaryEnsemble(
    n_parameter_samples=10,
    max_ensemble_size=3,
    randomly_selected_params=5,
)

In [81]:
#KNeighborsTimeSeries classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier(distance="euclidean")

In [127]:
#Catch22 classifier
from sktime.classification.feature_based import Catch22Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = Catch22Classifier(
    estimator=RandomForestClassifier(n_estimators=5),
    outlier_norm=True,
)

In [101]:
#Summary classifier
from sktime.classification.feature_based import SummaryClassifier
from sklearn.ensemble import RandomForestClassifier
classifier = SummaryClassifier(estimator=RandomForestClassifier(n_estimators=5))

In [104]:
#CanonicalIntervalForest classifier
from sktime.classification.interval_based import CanonicalIntervalForest
classifier = CanonicalIntervalForest(
    n_estimators=3, n_intervals=2, att_subsample_size=2
)

In [107]:
#DrCIF classifier
from sktime.classification.interval_based import DrCIF
classifier = DrCIF(n_estimators=3, n_intervals=2, att_subsample_size=2)

In [115]:
#Arsenal classifier
from sktime.classification.kernel_based import Arsenal
classifier = Arsenal(num_kernels=100, n_estimators=5)

In [119]:
#Shapelet classifier
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.sklearn import RotationForest
classifier = ShapeletTransformClassifier(
    estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=100,
    max_shapelets=10,
    batch_size=20,
)

In [128]:
# fit model
classifier.fit(X_train_tr, y_train)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\datatypes\_panel\_convert.py:713: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for _label, _series in multi_ind_dataframe.iteritems():  # noqa


Catch22Classifier(estimator=RandomForestClassifier(n_estimators=5),
                  outlier_norm=True)

In [141]:
# alternatively load model
# import trained classification model
with open('E:\Dokumente\Repos\kinematics-main\kinematics-main\model_Catch22_bird_plant_plane_30.pkl', 'rb') as f:
    clf = pickle.load(f)

In [142]:
# test the trained model
from pytictoc import TicToc
t = TicToc() #create instance of class

#sys.stdout = stdout
t.tic() #Start timer
y_pred = clf.predict(X_test)
t.toc() #Time elapsed since t.tic()

print(y_pred)
print(y_test)
#y_pred_proba = classifier.predict_proba(X_test)
score = accuracy_score(y_test, y_pred)

print(score)
#print(y_pred_proba)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sktime\datatypes\_panel\_convert.py:713: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for _label, _series in multi_ind_dataframe.iteritems():  # noqa


Elapsed time is 1.627024 seconds.
['Plant' 'Bird' 'Plant' 'Bird' 'Plant' 'Plane' 'Plant' 'Plane' 'Plane'
 'Plant' 'Bird' 'Plant' 'Bird' 'Bird' 'Plane' 'Bird' 'Plant' 'Plant'
 'Plane' 'Plane' 'Plant' 'Plant' 'Plant' 'Bird' 'Plane' 'Bird' 'Bird'
 'Plane' 'Plane' 'Plane' 'Plant' 'Bird' 'Plane' 'Plant' 'Plane' 'Plant'
 'Plane' 'Bird' 'Plant' 'Plant' 'Bird' 'Bird' 'Plant' 'Bird' 'Plane'
 'Plane' 'Plant' 'Plant' 'Plane' 'Plane' 'Bird' 'Bird' 'Plane' 'Plant'
 'Plane' 'Plant' 'Plane' 'Plant' 'Plant' 'Plant' 'Plant' 'Plane' 'Bird'
 'Plane' 'Plant' 'Plane' 'Plant' 'Bird' 'Plant' 'Plane' 'Bird' 'Bird'
 'Plant' 'Plant' 'Plant' 'Plant' 'Bird' 'Plant' 'Bird' 'Plant' 'Bird'
 'Plant' 'Plane' 'Plane' 'Bird' 'Plane' 'Plane' 'Plant' 'Bird' 'Bird'
 'Plant' 'Plane' 'Plane' 'Plane' 'Bird' 'Plane' 'Plant' 'Plane' 'Bird'
 'Plant' 'Bird' 'Bird' 'Plant' 'Plane' 'Bird' 'Plane' 'Bird' 'Plane'
 'Bird' 'Plant' 'Bird' 'Plane' 'Bird' 'Bird']
['Plant' 'Bird' 'Plant' 'Bird' 'Plant' 'Plane' 'Plant' 'Plane' 'Plane'
 'Pla

In [139]:
# save model
with open('E:\Dokumente\Repos\kinematics-main\kinematics-main\model_Catch22_bird_plant_plane_30.pkl','wb') as f:
    pickle.dump(classifier,f)

In [138]:
# not yet figured out how to reload models that are stored this way
classifier.save("E:\Dokumente\Repos\kinematics-main\kinematics-main\model_C22_bird_plant_plane_30")

<zipfile.ZipFile filename='E:\\Dokumente\\Repos\\kinematics-main\\kinematics-main\\model_C22_bird_plant_plane_30.zip' mode='r'>